# Project 4: **Build a Deep Research System**
Welcome to project 4! For this project, we shift our focus from tool use and agents to *reasoning* models. You will practice state‑of‑the‑art inference‑time scaling methods such as *Chain‑of‑Thought* prompting and *Tree‑of‑Thoughts*, and briefly explore high-levels of training reasoning models using techniques like **STaR**.


Finally, you will put everything together to build a *deep research agent* that can browse the web, reason over what it finds, and give structured answers.

## Learning Objectives  
* Apply common inference‑time scaling methods: **zero‑shot / few‑shot CoT, self‑consistency, sequential decoding, tree‑of‑thoughts**  
* Gain intuition for **training** reasoning‑capable models following **STaR** approach 
* Build a minimal **deep‑research agent** that combines step‑by‑step reasoning with live web search   
* Practice extending deep-search to a multi-agent system 

## Roadmap  
1. Environment setup  
2. Inference‑time scaling  
   2.1 Few‑shot & zero‑shot CoT  
   2.2 Self‑consistency
   2.3 Sequential revisions  
   2.4 Tree‑of‑Thought
3. STaR for training models for reasoning  
4. Deep-research agent  
5. (Optional) Multi-agent deep-research

# 1‑ Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yaml && conda activate deep_research

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=deep_research --display-name "deep_research"
```
Once this is done, you can select "deep_research" from the Kernel → Change Kernel menu in Jupyter or VS Code.

## 1.2 Ollama setup

In this project we use the `llama3.2:3b` and `deepseek-r1:8b` models. You can try other smaller or larger reasoning LLMs such as `qwen2.5:3b-instruct` or `phi4-mini` to compare performance. Explore available models here: https://ollama.com/library.

```bash
ollama pull llama3.2:3b
ollama pull deepseek-r1:8b
# Additional small reasoning models to compare
# ollama pull qwen2.5:3b-instruct
# ollama pull phi4-mini

```

`ollama pull` downloads the model so you can run it locally without API calls.

---  
# 2‑ Inference‑time scaling

Inference-time scaling refers to techniques that make an existing model reason better without retraining it. Instead of changing the model’s weights, we achieve reasoning capability by adjusting how we prompt, sample, or aggregate LLM's outputs.

In this section, we’ll explore several inference-time strategies that improve reasoning quality using a non-reasoning base model. You will experiment with and compare methods such as:

- Few-shot Chain-of-Thought (CoT)
- Zero-shot CoT
- Self-consistency
- Sequential revision
- Tree-of-Thoughts (ToT)

### 2.1: Few‑Shot CoT
Few-shot prompting helps a model reason by showing one or multiple examples before asking a new question. By observing the pattern of reasoning and final answers, the model learns how to structure its own reasoning process on the new input.

In this exercise, you will create a prompt that includes a few example Q&A pairs demonstrating step-by-step reasoning. Then, you will feed a new question and see the model’s output.

In [19]:
# Step 1: Write a few examples showing reasoning steps
# Step 2: Write your new question
# Step 3: Concatenate examples + new question into a single prompt
# Step 4: Call your Ollama or OpenAI client to get a response from llama3.2:3b # e.g., client.chat.completions.create(...)
# Step 5: Print the final answer

from openai import OpenAI

"""
YOUR CODE HERE (~10 lines of code)
"""
EXAMPLES = """
Q: What is the 9th Fibonacci number?
A: The Fibonacci sequence starts with 0 and 1. Each next number is found by adding the previous 2 numbers. So the sequence goes:
* 0
* 1
* 0 + 1 = 1
* 1 + 1 = 2
* 1 + 2 = 3
* 2 + 3 = 5
* 3 + 5 = 8
* 5 + 8 = 13
* 8 + 13 = 21

So the 9th Fibonacci number is 21.
"""

USER_QUESTION = """
Q: What is the 4th number in an arithmetic sequence with a common difference of 1 and a first term of 2?
A: """


llm = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")

response = llm.chat.completions.create(
    model="llama3.2:3b",
    messages=[
        {"role": "system", "content": EXAMPLES},
        {"role": "user", "content": USER_QUESTION},
        ]
)

print(response.choices[0].message.content)

To find the nth term of an arithmetic sequence, we use the formula:

an = a1 + (n - 1)d

where:
a1 = the first term (2 in this case)
d = the common difference (1 in this case)
n = the term number

Since you want to know the 4th term, plug in n = 4:

a4 = 2 + (4 - 1) * 1
= 2 + 3 * 1
= 2 + 3
= 5

So the 4th number in the sequence is 5.


### (Optional) Few-shot CoT on GPT2
GPT-2 is a pre-trained language model without instruction tuning. It continues text rather than answering questions. In this section, you'll try the exact same CoT pattern on GPT-2 and observe what happens. The goal is to test whether few-shot CoT alone can elicit structured reasoning from a non-chat LLM.

In [20]:
import os
import torch
from transformers import pipeline

# Step 1: Load GPT-2 text-generation from huggingface (https://huggingface.co/docs/transformers/en/model_doc/gpt2)
# Step 2: Write 1–2 few-shot reasoning examples (short, explicit steps + final answer in your own unique format)
# Step 3: Append a new test question after the examples to form one prompt string
# Step 4: Generate 1–3 completions with different decoding settings (e.g., greedy vs. top-k)
# Step 5: Print raw outputs; check if steps are followed and if the final answer is correct

"""
YOUR CODE HERE (~12-15 lines of code)
"""
pipeline = pipeline(task="text-generation", model="openai-community/gpt2", dtype=torch.float16, device=0)
pipeline(EXAMPLES + USER_QUESTION, max_new_tokens=20, do_sample=True, temperature=0.7, top_k=10, num_return_sequences=3)

Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '\nQ: What is the 9th Fibonacci number?\nA: The Fibonacci sequence starts with 0 and 1. Each next number is found by adding the previous 2 numbers. So the sequence goes:\n* 0\n* 1\n* 0 + 1 = 1\n* 1 + 1 = 2\n* 1 + 2 = 3\n* 2 + 3 = 5\n* 3 + 5 = 8\n* 5 + 8 = 13\n* 8 + 13 = 21\n\nSo the 9th Fibonacci number is 21.\n\nQ: What is the 4th number in an arithmetic sequence with a common difference of 1 and a first term of 2?\nA: 0000000000000000\n\nQ: What is the 5th Fibonacci number?\n\nA: '},
 {'generated_text': '\nQ: What is the 9th Fibonacci number?\nA: The Fibonacci sequence starts with 0 and 1. Each next number is found by adding the previous 2 numbers. So the sequence goes:\n* 0\n* 1\n* 0 + 1 = 1\n* 1 + 1 = 2\n* 1 + 2 = 3\n* 2 + 3 = 5\n* 3 + 5 = 8\n* 5 + 8 = 13\n* 8 + 13 = 21\n\nSo the 9th Fibonacci number is 21.\n\nQ: What is the 4th number in an arithmetic sequence with a common difference of 1 and a first term of 2?\nA: \xa0The 4th Fibonacci number is the first ter

### 2.2: Zero‑Shot Chain‑of‑Thought
Zero-shot CoT encourages the model to reason without examples by adding a short cue such as “Let’s think step by step.” This simple phrase often activates the model’s latent reasoning ability even when no demonstrations are provided. It serves as a baseline to compare with few-shot and other inference-time scaling methods.

In [21]:
from openai import OpenAI

# Step 1: Write the question and a zero-shot CoT cue (e.g., "Let's think step by step.")
# Step 2: Build a single prompt string that includes brief role guidance plus the question
# Step 3: Call your Ollama or OpenAI client to get a response from llama3.2:3b  # e.g., client.chat.completions.create(...)
# Step 4: Print the chain and the final answer

"""
YOUR CODE HERE (~10 lines of code)
"""
response = llm.chat.completions.create(
    # model="llama3.2:3b",
    model="deepseek-r1:8b",
    messages=[
        {"role": "user", "content": USER_QUESTION + "Let's think step by step."},
    ]
)

print(response.choices[0].message.content)

The fourth number in an arithmetic sequence with a first term of 2 and a common difference of 1 can be found using the formula for the nth term:

a_n = a_1 + (n - 1) * d

where:
- a_1 = 2 (first term)
- d = 1 (common difference)
- n = 4 (term to find)

Substitute the values into the formula:

a₄ = 2 + (4 - 1) * 1  
a₄ = 2 + (3) * 1  
a₄ = 2 + 3  
a₄ = 5

This result can be verified by listing out each term sequentially:
- First term: a₁ = 2
- Second term: a₂ = a₁ + d = 2 + 1 = 3
- Third term: a₃ = a₂ + d = 3 + 1 = 4
- Fourth term: a₄ = a₃ + d = 4 + 1 = 5

Thus, the fourth number is 5.


### 2.3 Self‑Consistency
Self-consistency enhances reasoning accuracy by sampling multiple independent reasoning paths for the same question instead of relying on a single deterministic answer. Each run may follow a slightly different logical chain, and the diversity helps correct individual mistakes. After generating several reasoning traces, you then aggregate the final answers using majority voting.

This approach is especially useful when tasks involve multi-step reasoning or arithmetic, where single-path outputs may be incorrect.

In [8]:
from openai import OpenAI
import re, collections, string

client = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")
MODEL = "llama3.2:3b"

def cot_answer(question, temperature=1.0):
    # Generate a step-by-step reasoning chain for the given question and extract the final answer.
    """
    YOUR CODE HERE (~10 lines of code)
    """
    response = client.chat.completions.create(
        model = MODEL,
        messages=[
            {"role": "user", "content": question + "\nA: Let's think step by step."},
        ],
        temperature=temperature,
    )

    return response.choices[0].message.content

def self_consistent(question, n=10):
    # Run multiple reasoning chains and select the most frequent final answer by majority voting.
    """
    YOUR CODE HERE (~12 lines of code)
    """
    responses = []
    response_counts = {}
    greatest_key = None
    for _ in range(n):
        response = cot_answer(question, 1.0)
        print("--------------------------------")
        print(response)
        print("--------------------------------")
        responses.append(response)
        # take the last 4 characters of the response, remove punctuation and whitespace and add
        # it to a hashmap of response -> count.  If the current value is greater than the greatest so far,
        # replace it with the current value and replace the greatest key with the current key.
        last_4 = response[-4:]
        last_4 = last_4.translate(str.maketrans('', '', string.punctuation))
        last_4 = last_4.strip()
        response_counts[last_4] = response_counts.get(last_4, 0) + 1
        if response_counts[last_4] > response_counts.get(greatest_key, 0):
            greatest_key = last_4
    # winner, counter = collections.Counter(responses).most_common(1)[0]
    winner, counter = greatest_key, response_counts[greatest_key]
    return winner, counter



question = "What is the square root of 144?"
winner, counter = self_consistent(question)
print("="*100)
print("Votes:", counter)
print("Chosen answer:", winner)
print("="*100)

--------------------------------
To find the square root of 144, we need to determine the number that multiplies itself to give us 144.

Step 1: Start with a perfect square like 12² = 144.

Step 2: Since we have found our starting point, 144 equals (12) squared. We can say our answer is 12 because it’s equal to (square root of (12)) squared.

Therefore, the square root of 144 is 12 .
--------------------------------
--------------------------------
To find the square root of 144, we need to determine the number that multiplied by itself equals 144.

Step 1: We know that 12 * 12 = 144.

Step 2: Therefore, the square root of 144 is 12, because 12 multiplied by itself results in 144.
--------------------------------
--------------------------------
To find the square root of 144, we can start by looking for perfect squares that are close to 144.

We know that:

* 11^2 = 121 (which is less than 144)
* 12^2 = 144 (which is exactly 144!)
* 13^2 = 169 (which is greater than 144)

Since 12^2 e

### 2.4: Sequential Revision

Sequential revision iteratively improves an answer by generating a first draft, critiquing it, and producing revised drafts that condition on prior answers. Each round should be short and focused, so improvements accumulate without drifting from the question.

In [33]:
MODEL = "llama3.2:3b"

def sequential_revision(question: str, max_steps: int = 3) -> str:
    # Generate an initial draft answer, then iteratively refine it by conditioning each revision on the previous one.
    # Step 1: Ask the model to produce the first draft for the given question
    # Step 2: Loop for max_steps-1 times, each time feeding the last draft back to the model with a request to revise
    # Step 3: Print each draft to observe how the answer evolves
    # Step 4: Return the final improved draft
    """
    YOUR CODE HERE (~20 lines of code)
    """
    response = client.chat.completions.create(
        model = MODEL,
        messages=[
            {"role": "user", "content": question + "\nA: Let's think step by step."},
        ],
    )
    for _ in range(max_steps - 1):
        last_response = response.choices[0].message.content
        print("--------------------------------")
        print(last_response)
        print("--------------------------------")
        response = client.chat.completions.create(
            model = MODEL,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that revises answers.  Here's the last answer: "},
                {"role": "user", "content": last_response},
            ],
        )
    return response.choices[0].message.content


# Step 1: Define a question that benefits from multi-step reasoning
# Step 2: Call sequential_revision(question, max_steps)
# Step 3: Print the final output
"""
YOUR CODE HERE (~5 lines of code)
"""
print(sequential_revision(question))

--------------------------------
To find the square root of 144, we can start by thinking about what a perfect square is. A perfect square is a number that can be expressed as the product of an integer multiplied by itself.

For example, some perfect squares are:

- 1 = 1 × 1
- 4 = 2 × 2
- 9 = 3 × 3

Now, let's try to find two integers whose product is 144. If we start multiplying numbers, we can see that:

- 12 × 12 = 144

Wow, it looks like 12 multiplied by itself gives us 144!

Therefore, the square root of 144 is... 

√144 = 12
--------------------------------
--------------------------------
The correct answer is: √144 = 12

However, to complete the explanation, we can also express this as:

Since 12 multiplied by itself results in 144, and the reverse operation (taking the inverse or finding the "square root") is needed to go back to just 12, it's more conventional to write the square root of a number as both its positive value and its negative equivalent.

Therefore, another way

### 2.5 Tree‑of‑Thoughts
Tree-of-Thoughts reframes reasoning as a search process rather than a single forward chain.
Instead of producing one linear sequence of thoughts, the model generates multiple candidate thoughts at each step, evaluates their promise, and then expands only the best few. This allows exploration of different reasoning paths before committing to a final answer, similar to how humans brainstorm, prune, and refine ideas.


In this section, you’ll experiment with two simplified versions of ToT:
1. Word Ladder puzzle solver: a small example where each “thought” is a candidate word transition.
2. Generic ToT search (depth 2, width 2): a minimal logic to expand, evaluate, and select reasoning branches

In [9]:
###### Word Ladder Puzzle ##########
import string


def neighbors(word, vocabulary):
    # Generate all valid one-letter mutations of 'word' that exist in 'vocabulary' and return them.
    """
    YOUR CODE HERE (~6-8 lines)
    """
    neighbors = []
    for i in range(len(word)):
        for c in string.ascii_lowercase:
            if c != word[i]:
                neighbor = word[:i] + c + word[i+1:]
                if neighbor in vocabulary:
                    neighbors.append(neighbor)
    return neighbors

def tree_of_thought(start, goal, vocab, max_depth=5, beam_width=4):
    # Search over partial thoughts (paths) using a small beam.
    # Step 1: Initialize the frontier with a single path [start]
    # Step 2: For each depth, expand each path by one neighbor from 'neighbors'
    # Step 3: Score paths by edit distance between last word and 'goal' (smaller is better)
    # Step 4: Keep the top 'beam_width' paths and stop early if any reaches 'goal'
    # Step 5: Return the best goal-reaching path or None
    """
    YOUR CODE HERE (~14-18 lines)
    """
    old_frontier = [[start]]
    already_searched = set()
    already_searched.add(start)
    for _ in range(max_depth):
        print("old_frontier", old_frontier)
        new_frontier = []
        for path in old_frontier:
            neighbs = neighbors(path[-1], vocab)
            for nb in neighbs:
                if nb in already_searched:
                    continue
                already_searched.add(nb)
                new_frontier.append(path + [nb])

        scores = {}
        for path in new_frontier:
            edit_distance = sum(1 for a, b in zip(path[-1], goal) if a != b)
            if edit_distance == 0:
                return path
            scores["~".join(path)] = edit_distance
        old_frontier = sorted(new_frontier, key=lambda x: scores["~".join(x)])[:beam_width]
    print("old_frontier", old_frontier)
    return None


vocab = {"hit","dot","cog","log","dog","lot","lit","hot"}
print(tree_of_thought("hit", "cog", vocab)) # one candidate solution: ['hit', 'hot', 'dot', 'dog', 'cog']


old_frontier [['hit']]
old_frontier [['hit', 'hot'], ['hit', 'lit']]
old_frontier [['hit', 'hot', 'dot'], ['hit', 'hot', 'lot']]
old_frontier [['hit', 'hot', 'dot', 'dog'], ['hit', 'hot', 'lot', 'log']]
['hit', 'hot', 'dot', 'dog', 'cog']


In [16]:
###### Generic ToT Search ##########

import re
from openai import OpenAI

MODEL = "llama3.2:3b"
client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")

def propose_thoughts(question, state, k=2):
    # Propose up to k next “thoughts” that extend the current partial solution/state.
    # Steps: build a short prompt with problem + current state; call your client with n=k. Then return a list of stripped strings (≤ k).
    """
    YOUR CODE HERE (~8-10 lines)
    """
    results = []
    for _ in range(k):
        response = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": question + "\n" + state}],
            temperature=1.0,
        )
        results.append(response.choices[0].message.content.strip())
    return results


def score_state(question, state):
    # Score how promising a partial solution is on a 1–10 scale (higher is better).
    # Steps: build a rating prompt; call the model; parse the first integer 1–10;
    """
    YOUR CODE HERE (~8-10 lines)
    """
    response = client.chat.completions.create(
        model = MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that scores the quality of a partial solution to a problem on a scale of 1 to 10.  Only return the score, no other text.  Here's the problem and the current state:"},
            {"role": "user", "content": question + "\n" + state},
        ],
    )
    result = response.choices[0].message.content
    print("--------------------------------")
    print(result)
    print("--------------------------------")
    return int(result)


def tree_of_thoughts(question, depth=2, width=2):
    # Run a tiny ToT search: expand states with propose_thoughts, score with score_state, keep top-k at each depth.
    # Steps: initialize frontier=[("", 0)]; for each depth, expand each state with k=width thoughts; score each; sort by score desc; keep top 'width'; return best state and score.
    """
    YOUR CODE HERE (~12-16 lines)
    """
    old_frontier = [("", 0)]
    for _ in range(depth):
        new_frontier = []
        for thought_node, score in old_frontier:
            new_thought_nodes = propose_thoughts(question, thought_node, width)
            new_frontier.extend([(new_thought_node, score_state(question, new_thought_node)) for new_thought_node in new_thought_nodes])
        old_frontier = sorted(new_frontier, key=lambda x: x[1], reverse=True)[:width]
    return old_frontier[0]

question = "Design a plan for a weekend science workshop for 12-year-olds."
solution, score = tree_of_thoughts(question)

print(f"Best solution (score {score}):\n{solution}")

--------------------------------
8
--------------------------------
--------------------------------
8
--------------------------------
--------------------------------
8
--------------------------------
--------------------------------
8
--------------------------------
--------------------------------
9
--------------------------------
--------------------------------
8
--------------------------------
Best solution (score 9):
Here's an updated plan for a fun and engaging weekend science workshop for 12-year-olds, incorporating the suggestions provided:

**Workshop Title:** "Mystery of the Universe"

**Objective:** To spark curiosity, creativity, and critical thinking in young scientists, while introducing them to fundamental concepts in physics, chemistry, biology, and earth sciences.

**Age Group:** 12-year-olds (6th-7th grade)

**Duration:** Saturday and Sunday, 9:30 AM - 5:00 PM

**Venue:** Science museum, community center, or outdoor space with minimal supervision required.

**A

---  
# 3‑ Training Models for Reasoning

### 3.1: CoT Training
Chain-of-Thought (CoT) training conditions the model on explicit rationales during fine-tuning. Instead of teaching the model to output only the final answer, we train on (question, rationale, answer) so the model learns to internalize multi-step reasoning patterns. A practical recipe is STaR (Self-Taught Reasoner), which uses a stronger teacher model to bootstrap rationales that a smaller student can learn from.

For tasks that require multi-hop reasoning, models fine-tuned on rationales often achieve higher accuracy and are more stable at inference time than models trained on direct answers only. 

Training a full language model is beyond the scope of this notebook, but here is the high-level workflow followed by a short pseudocode:
- Collect questions: Prepare a dataset of questions and correct answers.
- Generate rationales: Use a strong LLM to produce step-by-step reasoning ending with the correct answer.
- Filter and clean: Discard incorrect or low-quality rationales.
- Prepare training data: Format triples (question, rationale, answer) for supervised fine-tuning.
- Fine-tune: Fine-tune the LLM on rationales.
- Iterate: Refine prompts, improve data quality, and retrain for stronger reasoning.

In [ ]:
# Pseudocode (STaR loop)
# for round in 1 ... iters:
    # STEP 1: self-generate reasoning (teacher creates rationale + answer)
    # STEP 2: keep only correct, high-quality traces
    # STEP 3: fine-tune student on (question, rationale, answer) data

### 3.2: ORM vs PRM + RL
Training a Reward Model (RM) allows large language models to be improved through reinforcement learning (RL). Instead of fine-tuning directly on examples, we train a separate model that can score or rank model outputs, and use those scores as feedback signals to refine the policy model.

Two main reward modeling approaches are ORM (predicts a scalar reward for the final answer) and PRM (evaluates the reasoning steps instead of just the outcome)



| Approach | Typical loss | When to use |
|-----------|-------------|-------------|
|*Outcome Reward Model* | Predict scalar reward | Easy to collect training data using verifiers |
|*Process Reward Model* | Predict rewards per step | Difficult to collect training data but more accurate |
| *RLHF* | Use RM as reward in **RL** fine‑tuning | Aligns policy with human signals | Aligns model policy with human or synthetic preferences




In [ ]:
# for round = 1 ... iters:
    # STEP 1:  Generate reasoning
        # sample a minibatch of questions
        # policy roll‑out (actions + log‑probs)
    # STEP 2:  Score the trajectory
        # ORM: scalar reward for the final answer / PRM: scalar reward for the thought process
    # STEP 3:  Reinforce the policy (PPO)

---  
# 4‑ A Deep Research Agent

A deep-research agent pairs a reasoning model (e.g., deepseek-r1) with external tools for web search and retrieval. We will follow the ReAct pattern: the model writes short thoughts, decides when to call tools, reads observations, and continues reasoning until it can answer or reaches a step limit.

We now combine a **search tool** with a reasoning model (e.g., `deepseek-r1`) in a multi-step setup. We follow the *ReAct* pattern (reason → tool → observation):

1. The model reasoins and decides to use tools
2. The agent searches and feed condensed snippets back as context
3. Iterate until the model answers or hits a step limit

We use `AgentType.OPENAI_FUNCTIONS`, which hides the loop inside the LangChain agent.

In [17]:
from ddgs import DDGS
from langchain.tools import Tool

def ddg_search(query: str, k: int = 5) -> str:
    # Use DDGS to run a simple web search and return joined snippets.
    """
    YOUR CODE HERE (~6 lines of code)
    """
    results = DDGS().text(query, max_results=k)
    return "\n".join([result['title'] + "\n" + result['href'] + "\n" + result['body'] for result in results])


search_tool = Tool(
    name="DuckDuckGo Search",
    func=ddg_search,
    description="Search the public web. Input: a plain English query. Returns: concatenated snippets."
)


In [18]:
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOllama

MODEL = "deepseek-r1:8b"
question = "What are the best resources to learn machine learning in 2025?"

# Step 1: Initialize the reasoning model via ChatOllama
"""
YOUR CODE HERE (1 line of code)
"""
llm = ChatOllama(model=MODEL, temperature=0)


# Step 2: Build the agent with tool access (DuckDuckGo Search) and function-calling interface (initialize_agent)
"""
YOUR CODE HERE (1 line of code)
"""
tools = [search_tool]
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# Step 3: Ask a query and let the agent search + reason to produce an answer
"""
YOUR CODE HERE (2 lines of code)
"""
agent.invoke(question)

/var/folders/v4/rz21fv5s5fn_4nv27jvlbh3m0000gn/T/ipykernel_73452/1378146609.py:11: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=MODEL, temperature=0)
/var/folders/v4/rz21fv5s5fn_4nv27jvlbh3m0000gn/T/ipykernel_73452/1378146609.py:19: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `M



> Entering new AgentExecutor chain...

Okay, let's break down the best resources to learn Machine Learning (ML) in a way that anticipates its evolution towards 2025. The core concepts will remain relevant, but tools and applications are constantly evolving.

The "best" resource depends heavily on your starting point (beginner, intermediate, advanced), background knowledge, learning style (visual, hands-on, theoretical), and goals. Here's a categorized overview focusing on foundational skills that will be essential in 2025:

## I. Foundational Concepts & Python Programming

1.  **Python Mastery:** ML heavily relies on Python libraries.
    *   **Resource:** "Python for Data Analysis" by Wes McKinney (book), official NumPy/ Pandas/ Scikit-learn documentation, interactive platforms like Google's Numerical Methods in Finance and Economics Specialization (uses SciPy) or introductory courses focusing specifically on data manipulation with these tools. Understanding Python basics, control f

{'input': 'What are the best resources to learn machine learning in 2025?',
 'output': '\nOkay, let\'s break down the best resources to learn Machine Learning (ML) in a way that anticipates its evolution towards 2025. The core concepts will remain relevant, but tools and applications are constantly evolving.\n\nThe "best" resource depends heavily on your starting point (beginner, intermediate, advanced), background knowledge, learning style (visual, hands-on, theoretical), and goals. Here\'s a categorized overview focusing on foundational skills that will be essential in 2025:\n\n## I. Foundational Concepts & Python Programming\n\n1.  **Python Mastery:** ML heavily relies on Python libraries.\n    *   **Resource:** "Python for Data Analysis" by Wes McKinney (book), official NumPy/ Pandas/ Scikit-learn documentation, interactive platforms like Google\'s Numerical Methods in Finance and Economics Specialization (uses SciPy) or introductory courses focusing specifically on data manipulati

# Optional (Multi-agent Deep Research)
Instead of a single multi-step agent, you can design multiple collaborating agents such as a Planner, Searcher, Summarizer, and Verifier that pass information and refine each other’s outputs. This setup improves robustness, diversity of reasoning, and division of labor.

Try building a simple setup with 2–3 agents that share goals and messages, for example Planner → Researcher → Writer.

In [21]:
from concurrent.futures import ThreadPoolExecutor


def parallel_research(query, n=3):
    # Run n independent research runs in parallel and return their answers.
    # Steps: use ThreadPoolExecutor; submit n calls to your agent/search pipeline; gather results in order.
    """
    YOUR CODE HERE
    """
    with ThreadPoolExecutor(max_workers=n) as executor:
        futures = [executor.submit(agent.invoke, query) for _ in range(n)]
        answers = [future.result() for future in futures]
    return answers

answers = parallel_research("What are the best resources to learn ML in 2025?")



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Okay, since we're looking ahead to resources potentially relevant or emerging by 2025, it's crucial to focus on foundational concepts and platforms with longevity, while also keeping an eye on rapidly evolving areas like foundation models (large language models, vision transformers etc.) and MLOps. Here’s a breakdown of likely best resources:

## I. Foundational Knowledge (Essential & Evergreen)

1.  **Core Concepts Books:**
    *   **"Pattern Recognition and Machine Learning" by Christopher Bishop:** A comprehensive textbook covering the fundamentals.
    *   **"The Elements of Statistical Learning" by Hastie, Tibshirani, Friedman et al.:** The bible for statistical learning theory (available online free). Focuses on understanding principles.
    *   **"Deep Learning" by Goodfellow, Bengio, and Courville:** The definitive deep learning book. Requires strong math bac

In [25]:

for i,a in enumerate(answers,1):
    print(f"[Run {i}] {a['output'][:200]}…")

[Run 1] 
Okay, since we're looking ahead to resources potentially relevant or emerging by 2025, it's crucial to focus on foundational concepts and platforms with longevity, while also keeping an eye on rapidl…
[Run 2] 
Okay, since we're looking ahead to resources potentially relevant or emerging by 2025, it's crucial to focus on foundational concepts and platforms with longevity, while also keeping an eye on rapidl…
[Run 3] 
Okay, since we're looking ahead to resources potentially relevant or emerging by 2025, it's crucial to focus on foundational concepts and platforms with longevity, while also keeping an eye on rapidl…


## 🎉 Congratulations!

* Practised various inference‑time reasoning methods
* Gained intuition about training reasoning models
* You have built a **deep-research agent**: reasoning model like deep-seek r1 + ReAct-style agent + tool use (web search)
* Try adding more tools, and extending the deep-research to a multi-agent system: many agents researching web in parallel.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.